In [1]:
# IMPORTS
from geofluids import *

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd


In [2]:
# PARAMETERS
IDX_MIN = 28
IDX_MAX = 52

SHEET_ID = '1jU6eor_WTvCt3tLzQbysIg0AH-8zRwek_B0aUV76O1k'
SHEET_NAME = 'Parameters'
OUTPUT_PATH = 'output/02_blue/'
IMAGE_PATH = 'images/02_blue/'
C = 2


In [3]:
# FUNCTIONS
def update_line(t_id, label_list, lines):
    labels = [solver.color_A, solver.color_B, solver.color_C]
    
    for line, label in zip(lines, labels):
        idx = (label_list[t_id] == label)
        line.set_data(pos_x_list[t_id, idx], pos_y_list[t_id, idx])

In [4]:
# SETUP
url = f'https://docs.google.com/spreadsheets/d/{SHEET_ID}/gviz/tq?tqx=out:csv&sheet={SHEET_NAME}'
df = pd.read_csv(url)
df.head()

,dx,mean_lnk,variance_lnk,correlation_length,random_seed,perm_field,long_disp,trans_disp,num_particles,species_1,species_2,species_3,t_steps,save_interval,completed
0,1,-12,1.5,10,0,10,2,0.25,10000,C0,C9,C3,5000,8,yes
1,1,-12,0.5,15,0,10,2,0.50,10000,C0,C9,C3,5000,8,yes
2,1,-12,0.5,20,0,10,2,0.50,10000,C0,C9,C3,5000,8,yes
3,1,-12,0.5,25,0,10,2,0.50,10000,C0,C9,C3,5000,8,yes
4,1,-12,1.0,25,0,10,2,0.50,10000,C0,C9,C3,5000,8,yes


In [6]:
# Single threaded execution [thread safety]
for idx in range(IDX_MIN, IDX_MAX+1):
    # Selecting parameters
    s = df.loc[idx-C]
    
    # Setting up permeability field
    lnk = Perm2D(s['mean_lnk'], s['variance_lnk'], s['dx'], s['correlation_length'], s['random_seed'])
    
    # Plotting Perm field (1)
    plt.figure(figsize=(10, 4))
    plt.pcolormesh(lnk.T)
    plt.colorbar(orientation='horizontal', shrink=0.7)
    plt.gca().set_aspect(1)
    plt.savefig(f'{IMAGE_PATH}index_{idx}_perm_field.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # Setting up solver
    solver = RWPT_solver()
    solver.set_permeability_field(s['perm_field']**lnk)
    p, mesh = solver.solve_flow_field()
    solver.generate_velocity_interpolators()
    x_space = np.linspace(0.0, solver.Lx, 150)
    y_space = np.linspace(0.0, solver.Ly, 50)
    x_grid, y_grid = np.meshgrid(x_space, y_space, indexing='xy')
    u_grid = solver.interpolator_u(x_grid, y_grid)
    v_grid = solver.interpolator_v(x_grid, y_grid)
    u_norm = np.sqrt(u_grid**2 + v_grid**2)
    x_center = mesh.cellCenters.value[0]
    y_center = mesh.cellCenters.value[1]
    solver.set_longitudinal_dispersivity(s['long_disp'])
    solver.set_transversal_dispersivity(s['trans_disp'])
    solver.generate_dispersivitiy_interpolators()
    solver.set_num_particles(s['num_particles'])
    solver.set_specie_labels(s['species_1'], s['species_2'], s['species_3'])
    solver.set_time_steps(s['t_steps'])
    solver.set_save_interval(s['save_interval'])
    solver.set_initial_particle_position()

    # Plotting velocity and permeability (2)
    plt.figure(figsize=(10, 4))
    plt.streamplot(x_grid, y_grid, u_grid, v_grid, density=[6.0, 1.5], color=np.log(u_norm), cmap='Greys')
    plt.tricontourf(x_center, y_center, lnk.T.flatten(), cmap='viridis')
    plt.gca().set_aspect(1)
    plt.savefig(f'{IMAGE_PATH}index_{idx}_velocity_field.png', dpi=300, bbox_inches='tight')
    plt.close()

    # Plotting pressure (3)
    plt.figure(figsize=(10, 4))
    plt.tricontourf(x_center, y_center, p)
    plt.gca().set_aspect(1.0)
    plt.savefig(f'{IMAGE_PATH}index_{idx}_pressure_field.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # Plotting initial conditions (4)
    plt.figure(figsize=(10, 4))
    plt.streamplot(x_grid, y_grid, u_grid, v_grid, density=[6.0, 1.5], 
                color=np.log(u_norm), cmap='Greens', zorder=-1)
    plt.scatter(solver.pos_x, solver.pos_y, c=solver.label, s=8)
    plt.gca().set_aspect(1)
    plt.savefig(f'{IMAGE_PATH}index_{idx}_initial_conditions.png', dpi=300, bbox_inches='tight')
    plt.close()
    
    # Reading particle simulation data
    pos_x_list, pos_y_list, label_list = np.load(f'{OUTPUT_PATH}index_{idx}.npz').values()
    
    # Plotting particle trajectories (5)
    t_id = -1
    idx_A = (label_list[t_id] == solver.color_A)
    idx_B = (label_list[t_id] == solver.color_B)
    idx_C = (label_list[t_id] == solver.color_C)
    fig, ax = plt.subplots(1, 1, figsize=(10, 4))
    ax.streamplot(x_grid, y_grid, u_grid, v_grid, density=[6.0, 1.5], 
                color=np.log(u_norm), cmap='Greys_r', zorder=-1, linewidth=1)
    line_A = ax.plot(pos_x_list[t_id][idx_A], pos_y_list[t_id][idx_A], c=solver.color_A, 
                    marker='o', markersize=2, ls='None', alpha=0.3)
    line_B = ax.plot(pos_x_list[t_id][idx_B], pos_y_list[t_id][idx_B], c=solver.color_B,
                    marker='o', markersize=2, ls='None', alpha=0.3)
    try:
        line_C = ax.plot(pos_x_list[t_id][idx_C], pos_y_list[t_id][idx_C], c=solver.color_C,
                        marker='o', markersize=2, ls='None', alpha=0.1)
    except:
        print(f'No particles of type C for simulation {idx}')
    
    ax.set_xlim(0.0, solver.Lx)
    ax.set_ylim(0.0, solver.Ly)
    ax.set_aspect('equal')
    plt.savefig(f'{IMAGE_PATH}index_{idx}_particle_distribution.png', dpi=300, bbox_inches='tight')
    plt.close()


No particles of type C
No particles of type C
No particles of type C
No particles of type C
No particles of type C
